# Pretrained Network | Feature Extractor

El objetivo de esta sección es ver como implementar transfer-learning cuando tenemos una cantidad de datos disponible limitada, pero el dominio del objetivo es similar a la red que queremos usar.

Congelaremos la parte de extracción de features de la red VGG16 entrenada en el dataset IMAGENET, y agregaremos nuestro propio clasificador, para después reentrenar la red con nuestro pequeño dataset.

Otro de los objetivos será mostrar como preprocesar nuestros datos para conseguir entrenar una red.

Notesé que ya no calculamos MEAN y STD para normalizar aquí, si no que usamos los valores estandard (calculados por la comunidad) para rapidez

Además no usaremos data-augmentation aqui.

In [19]:
from torchvision import models, datasets, transforms
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchsummary import summary

#Obtenemos los pesos de la red VGG16 entrenada en el dataset IMAGENET

weights = models.VGG16_Weights.IMAGENET1K_V1

#Como vemos aqui observamos los valores a los que nuestro dataset debe ser normalizado
# y preprocesado para que la red pueda usarlo.

preprocess = weights.transforms()
print(preprocess)
IMAGENET_MEAN = preprocess.mean
IMAGENET_STD  = preprocess.std

print(IMAGENET_MEAN)
print(IMAGENET_STD)


ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
[0.485, 0.456, 0.406]
[0.229, 0.224, 0.225]


Como vemos arriba:

### crop_size=[224]

Es el tamaño final de la imagen que el modelo usa como entrada: 224×224 píxeles.
Se consigue aplicando transforms.CenterCrop(224) después del resize.

### resize_size=[256]

Antes del recorte, la imagen se redimensiona de forma que su lado más corto mida 256 píxeles, manteniendo su proporción.
Esto equivale a transforms.Resize(256)

También observamos la media y desviación estandard que usaremos para la normalización.

Este pipeline que hemos visto arriba, tenemos que replicarlo en nuestro dataset.

In [20]:


# Replicamos el pipeline de preprocesado de la red VGG16
base_tf = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR), # Redimensiona la imagen
    transforms.CenterCrop(224), # Recorta la imagen
    transforms.ToTensor(), # Convierte la imagen a un tensor
    transforms.Normalize(weights.transforms().mean, weights.transforms().std), # Normaliza la imagen
])

root = "./data/01"
train_ds = datasets.ImageFolder(f"{root}/train", transform=base_tf)
valid_ds = datasets.ImageFolder(f"{root}/valid", transform=base_tf)
test_ds  = datasets.ImageFolder(f"{root}/test",  transform=base_tf)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size=64, shuffle=False)


In [28]:
# Definimos GPU o CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Obtenemos el numero de clases de nuestro dataset
num_classes = len(train_ds.classes)

# Definimos la base de la red VGG16
base = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).to(device)


#Vamos a imprimir los parametros de la red VGG16
# Calculamos parámetros
trainable_params = sum(p.numel() for p in base.parameters() if p.requires_grad)
total_params     = sum(p.numel() for p in base.parameters())
frozen_params    = total_params - trainable_params

print(f"Trainable: {trainable_params:,} | Frozen: {frozen_params:,} | Total: {total_params:,}")



Trainable: 138,357,544 | Frozen: 0 | Total: 138,357,544


In [29]:

# Congelamos los parametros de la red VGG16
# Aqui le decimos que no calcule los gradientes para los parametros de la red VGG16
# Esto es importante, ya que si no lo hacemos, el modelo se entrenará desde cero
# y no aprovechará los pesos ya entrenados de la red VGG16-Imagenet
for p in base.features.parameters():
    p.requires_grad = False


# Vamos a volver a calcular los parametros de la red VGG16, para ver que se han congelado
# Calculamos parámetros
trainable_params = sum(p.numel() for p in base.parameters() if p.requires_grad)
total_params     = sum(p.numel() for p in base.parameters())
frozen_params    = total_params - trainable_params

print(f"Trainable: {trainable_params:,} | Frozen: {frozen_params:,} | Total: {total_params:,}")




Trainable: 123,642,856 | Frozen: 14,714,688 | Total: 138,357,544


Como vemos, los parametros entrenables, se han reducido, a continuación, vamos a quedarnos con toda la red, salvo el clasificador, ya que crearemos uno para nuestro caso de uso. (2 clases)

Como veremos en el output, al quitar el clasificador, ya no tenemos parametros entrenables, los que quedan de las capas de convolución estan congelados.

In [44]:
backbone = nn.Sequential(*(list(base.children())[:-1])) 

# Vemos que los parametros entrenables se han reducido a 0
trainable_params = sum(p.numel() for p in backbone.parameters() if p.requires_grad)
total_params     = sum(p.numel() for p in backbone.parameters())
frozen_params    = total_params - trainable_params

print(f"Trainable: {trainable_params:,} | Frozen: {frozen_params:,} | Total: {total_params:,}")

print(summary(backbone, (3, 224, 224)))

Trainable: 0 | Frozen: 14,714,688 | Total: 14,714,688
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]  

Para volver a tener parametros entreables desde 0, tendremos que añadir nuestra propia capa de clasificación a la red.

In [ ]:


# Definimos la capa de pooling y el clasificador
print()

head = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512*7*7, 64), # 512 es el numero de features de la ultima capa de convolución y 7*7 es el tamaño de la imagen de entrada.
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Dropout(0.5),
    nn.Linear(64, num_classes)
)

# Movemos la red a GPU o CPU
model = base.to(device)

# Obtenemos el numero de parametros entrenables y totales
sum_reqgrad = sum(p.requires_grad for p in model.parameters())
sum_total   = sum(1 for _ in model.parameters())
print(f"trainables: {sum_reqgrad}/{sum_total}")

# Definimos la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()